In [11]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash_table import DataTable
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input, State
poverty = pd.read_csv('../../data/poverty.csv',low_memory=False)
gini = 'GINI index (World Bank estimate)'
df = poverty[poverty['year'].eq(2000)&poverty['is_country']].filter(regex='Country Name|Income share')



In [12]:
%%capture
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = html.Div([
    html.Br(),
    dbc.Row([
        dbc.Col(lg=1,),
        dbc.Col([
            dbc.Label('Indicator:'),
            dcc.Dropdown(id='hist_indicator_dropdown',optionHeight=40,
                         value='GINI index (World Bank estimate)',
                         options=[{'label': indicator, 'value': indicator}
                                  for indicator in poverty.columns[3:54]]),
        ], lg=5),
        dbc.Col([
            dbc.Label('Years:'),
            dcc.Dropdown(id='hist_multi_year_selector',
                         multi=True,
                         value=[2015],
                         placeholder='Select one or more years',
                         options=[{'label': year, 'value': year}
                                  for year in poverty['year'].drop_duplicates().sort_values()]),
        ], lg=4),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col(lg=2),
        dbc.Col([
            html.Br(),
            dbc.Label('Modify number of bins:'),
            dcc.Slider(id='hist_bins_slider', 
                       dots=True, min=0, max=100, step=5, included=False,
                       marks={x: str(x) for x in range(0, 105, 5)}),
        ], lg=6)
        
    ]),

    dcc.Graph(id='indicator_year_histogram'),
    dbc.Row([
        dbc.Col(lg=2),
        dbc.Col([
            html.Div(id='table_histogram_output'),
        ], lg=7)
    ])

], style={'backgroundColor': '#E5ECF6'})

@app.callback(Output('indicator_year_histogram', 'figure'),
              Output('table_histogram_output', 'children'),
              Input('hist_multi_year_selector', 'value'),
              Input('hist_indicator_dropdown', 'value'),
              Input('hist_bins_slider', 'value'))
def display_histogram(years, indicator, nbins):
    if (not years) or (not indicator):
        raise PreventUpdate
    df = poverty[poverty['year'].isin(years) & poverty['is_country']]
    fig = px.histogram(df, x=indicator, facet_col='year', color='year', 
                       title=indicator + ' Histogram',
                       nbins=nbins,
                       facet_col_wrap=4, height=700)
    fig.for_each_xaxis(lambda axis: axis.update(title=''))
    fig.add_annotation(text=indicator, x=0.5, y=-0.12, xref='paper', yref='paper', showarrow=False)
    fig.layout.paper_bgcolor = '#E5ECF6'
    
    table = DataTable(columns = [{'name': col, 'id': col} 
                                 for col in df[['Country Name', 'year', indicator]].columns],
                      data = df[['Country Name', 'year', indicator]].to_dict('records'),

                      style_header={'whiteSpace': 'normal'},
                      fixed_rows={'headers': True},
                      virtualization=True,
                      style_table={'height': '400px'},
    
                      
                      sort_action='native',
                      filter_action='native',
                      export_format='csv',
                      style_cell={'minWidth': '150px'}),
                      
    html.Br(), html.Br(), html.Br()
    
    
    return fig, table

app.run_server(host='0.0.0.0',port=3000, height=900)

In [13]:
! echo 'Your app is running on :'$EDUCATIVE_LIVE_VM_URL':3000'

Your app is running on ::3000
